In [ ]:
feat_no = '106_new_'
stack_name='555_new_transaction'
stack_name='555_new_transaction_noTE'
#========================================================================
# Dataset Load
def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        
train = elo_load_data('train')
test = elo_load_data('test')
#========================================================================


%load_ext autoreload
%autoreload 2
#========================================================================
# Args
#========================================================================
learning_rate = 0.05
early_stopping_rounds = 150
num_boost_round = 10000
ignore_list = [key, target, 'merchant_id', 'purchase_date']

import gc
import sys
import numpy as np
import pandas as pd
import datetime
import glob
import os
HOME = os.path.expanduser('~')

sys.path.append(f'{HOME}/kaggle/data_analysis/model')
from params_lgbm import params_elo
sys.path.append(f'{HOME}/kaggle/data_analysis')
from model.lightgbm_ex import lightgbm_ex as lgb_ex

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

if model_type=='lgb':
    params = params_elo()
    params['learning_rate'] = learning_rate

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())


#========================================================================
# LGBM Setting
model_type='lgb'
metric = 'rmse'
fold=5
seed=1208
group_col_name=key
dummie=1
oof_flg=True
LGBM = lgb_ex(logger=logger, metric=metric, model_type=model_type, ignore_list=ignore_list)


train, test, drop_list = LGBM.data_check(train=train, test=test, target=target, encode='dummie', exclude_category=True)

ignore_list = [key, target, 'merchant_id', 'purchase_date']
cat_list = utils.get_categorical_features(df=train, ignore_list=ignore_list)
id_list = [col for col in train.columns if (col.count('_id') and col not in ignore_list) and not(col.count('TE'))]
cat_list += id_list

#========================================================================
# Train & Prediction Start
#========================================================================
import lightgbm as lgb

# TrainとCVのfoldを合わせる為、Train
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
raw_train = elo_load_data('train')
y = raw_train[target].values
tmp_train = raw_train.drop(target, axis=1).set_index(key)

folds = KFold(n_splits=fold, shuffle=True, random_state=seed)
kfold = folds.split(tmp_train, y)
train_id_list = np.array(tmp_train.index)
del raw_train, tmp_train, y
gc.collect()

y = train.set_index(key)[target]
display(train.head())
tmp_train = train.drop(target, axis=1).set_index(key)
use_cols = [col for col in train.columns if col not in ignore_list]

oof = pd.Series(np.zeros(len(train)), index=tmp_train.index, name='oof_pred')
score_list = []
prediction = np.zeros(len(test))

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    trn_idx = list(set(train_id_list[trn_idx]) & set(list(tmp_train.index)))
    val_idx = list(set(train_id_list[val_idx]) & set(list(tmp_train.index)))
    x_train, y_train = tmp_train[use_cols].loc[trn_idx, :], y.loc[trn_idx]
    x_val, y_val = tmp_train[use_cols].loc[val_idx, :], y.loc[val_idx]
    
    lgb_train = lgb.Dataset(data=x_train, label=y_train)
    lgb_eval = lgb.Dataset(data=x_val, label=y_val)
    
    lgbm = lgb.train(
        train_set=lgb_train,
        valid_sets=lgb_eval,
        params=params,
        verbose_eval=200,
        early_stopping_rounds=early_stopping_rounds,
        num_boost_round=num_boost_round,
        categorical_feature=cat_list
    )
    
    y_pred = lgbm.predict(x_val)
    prediction += lgbm.predict(test[use_cols])
    oof.loc[val_idx] = y_pred
    
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    logger.info(f"Validation {n_fold}: RMSE {score}")
    score_list.append(score)

#========================================================================
# Result
#========================================================================
cv_score = np.mean(score_list)
test['oof_pred'] = prediction/fold
stack_pred = pd.concat([oof, test.set_index(key)['oof_pred']], axis=0)
feature_num = len(use_cols)
# cv_feim.to_csv(f'../valid/{start_time[4:12]}_lgb_feat{feature_num}_CV{cv_score}_lr{learning_rate}.csv', index=False)

#========================================================================
# STACKING
logger.info(f'''
train_stack shape: {oof.shape}
test_stack shape: {test_pred.shape}
''')
utils.to_pkl_gzip(path=f"../stack/{stack_name}_lgb_CV{str(cv_score).replace('.', '-')}_feat{feature_num}", obj=stack_pred)
#========================================================================

In [ ]:
stack = utils.read_pkl_gzip('../stack/555_new_transaction_lgb_CV2-9005820682193173_feat34.gz')
stack_noTE = utils.read_pkl_gzip('../stack/555_new_transaction_noTE_lgb_CV2-9004063018367328_feat27.gz')
print(stack.shape)
print(stack_noTE.shape)
stack.head()

In [ ]:
stack_agg = stack.reset_index().groupby(key)['oof_pred'].agg({
    'oof1208_pred_mean@':'mean',
    'oof1208_pred_std@':'std',
    'oof1208_pred_max@':'max',
    'oof1208_pred_min@':'min',
    'oof1208_pred_skew@':'skew',
})
stack_agg['new_pred_max-min@'] =  stack_agg['oof1208_pred_max@'] - stack_agg['oof1208_pred_min@']

stack_noTE_agg = stack_noTE.reset_index().groupby(key)['oof_pred'].agg({
    'oof1208_pred_noTE_mean@':'mean',
    'oof1208_pred_noTE_std@':'std',
    'oof1208_pred_noTE_max@':'max',
    'oof1208_pred_noTE_min@':'min',
    'oof1208_pred_noTE_skew@':'skew',
})
stack_noTE_agg['new_pred_noTE_max-min@'] =  stack_noTE_agg['oof1208_pred_noTE_max@'] - stack_noTE_agg['oof1208_pred_noTE_min@']
print(stack_agg.shape)
print(stack_noTE_agg.shape)
stack_noTE_agg.head()

In [ ]:
base = utils.read_df_pkl('../input/base0*').set_index(key)
base = base.join(stack_agg, how='left').join(stack_noTE_agg, how='left')
prefix = ''

train_id = elo_load_data('train')[key].values
test_id = elo_load_data('test')[key].values
for col in base.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = base.loc[train_id, :][col].values, path=f'../features/1_first_valid/{feat_no}train_{prefix}{col}')
        utils.to_pkl_gzip(obj = base.loc[test_id, :][col].values, path=f'../features/1_first_valid/{feat_no}test_{prefix}{col}')